In [1]:
# Core libraries
import numpy as np
import pandas as pd

# System 
from pathlib import Path
import subprocess
from shutil import copy
import datetime

# Simulation
import tellurium as te
import roadrunner

In [2]:
%load_ext watermark

In [3]:
%watermark --iversions

pandas    0.22.0
numpy     1.14.3
roadrunner1.4.24
tellurium 2.1.3



### Setup paths and helper functions

In [4]:
models_path = Path("../data/models")
simulation_path = Path("../data/simulation_results")

In [5]:
def run_matlab_script(script_path):
    # script_path - Path object to the m script
    str_script = str((script_path).resolve())
    run_command = f""" -r "run('{str_script}');exit;" """

    matlab_run = subprocess.run(
        ["matlab", "-nodisplay", "-nosplash", "-nodesktop", run_command],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )

    # check for errors
    if matlab_run.returncode == 0:
        print("Run ended succesfully")
    else:
        print("Run ended with an error")

    return matlab_run.stdout.decode("UTF-8")

# Simulations of Chassagnole model

## Define the model and experimental setup

In [23]:
model = te.loadAntimonyModel(
    te.sbmlToAntimony(str(models_path / "original_files" / "Chassagnole2002.xml"))
)

## Run the simulations

In [ ]:
model.simulate(0, 1000, 20)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()


ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Chassagnole" / f"Chassagnole_result_WT.csv")

In [ ]:
for experiment in exp_list:
    # set the original state
    model.resetToOrigin()

    sample_id = experiment["sample_id"]
    print(f"Working on sample {sample_id}")

    for mod in experiment["modifications"]:
        # perform poor-mans 'knockout'
        model.setValue(mod, 0)

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 1000, 20)    

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()


    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv(simulation_path / "Chassagnole" / f"Chassagnole_result_{sample_id}.csv")

# Simulations of Millard model

## Define the model and experimental setup

In [6]:
# load as Antimony bypasses the lack of ability to change local parameters in native SBML model
model = te.loadAntimonyModel(
    te.sbmlToAntimony(
        str(models_path / "original_files" / "Millard_2017_MODEL1505110000_url.xml")
    )
)

## Run the simulations
Simulations are being performed as ODEs integration for 1e5 time units. Results after the end of simulation are being used as "steady-state" values.

In [20]:
# WT simulation
model.resetToOrigin()

# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4

model.simulate(0, 10000)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" /  "dilutions" / f"millard_dilution_02.csv")

In [21]:
# WT simulation
model.resetToOrigin()

# For Dilution rate approx. equal to 0.4 h-1
model.FEED = 0.73

model.simulate(0, 10000)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" /  "dilutions" / f"millard_dilution_04.csv")